# Natural Language Processing

## Importing the libraries

In [1]:
!pip install catboost

     |████████████████████████████████| 66.3MB 50kB/s 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [3]:
dataset = pd.read_csv('Ethos_Dataset_Binary.csv')

## Cleaning the texts

In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 998):
  review = re.sub('[^a-zA-Z]', ' ', dataset['comment'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Creating the Bag of Words model

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2700)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values
for i in range(0,998):
  if y[i] >= 0.5:
    y[i] = int(1)
  else:
    y[i] = int(0)

## Splitting the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Training the CatBoost model on the Training set

In [7]:
from catboost import CatBoostClassifier
classifier = CatBoostClassifier()
classifier.fit(X_train, y_train)

Learning rate set to 0.009101
0:	learn: 0.6927193	total: 56.5ms	remaining: 56.4s
1:	learn: 0.6923039	total: 67.3ms	remaining: 33.6s
2:	learn: 0.6920443	total: 77.4ms	remaining: 25.7s
3:	learn: 0.6915428	total: 87.6ms	remaining: 21.8s
4:	learn: 0.6910623	total: 97.7ms	remaining: 19.4s
5:	learn: 0.6904964	total: 108ms	remaining: 17.9s
6:	learn: 0.6900390	total: 118ms	remaining: 16.8s
7:	learn: 0.6896488	total: 129ms	remaining: 16s
8:	learn: 0.6892207	total: 139ms	remaining: 15.3s
9:	learn: 0.6886644	total: 149ms	remaining: 14.7s
10:	learn: 0.6881900	total: 159ms	remaining: 14.3s
11:	learn: 0.6878343	total: 169ms	remaining: 13.9s
12:	learn: 0.6874533	total: 179ms	remaining: 13.6s
13:	learn: 0.6868823	total: 190ms	remaining: 13.3s
14:	learn: 0.6866908	total: 203ms	remaining: 13.3s
15:	learn: 0.6862337	total: 213ms	remaining: 13.1s
16:	learn: 0.6859286	total: 223ms	remaining: 12.9s
17:	learn: 0.6855991	total: 233ms	remaining: 12.7s
18:	learn: 0.6852603	total: 243ms	remaining: 12.5s
19:	lear

## Making the Confusion Matrix

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[128  22]
 [ 54  46]]


0.696

In [9]:
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]
accuracy = (tp + tn)/(tp + tn + fp + fn)
precision = (tp)/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*precision*recall/(precision + recall)

In [10]:
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 Score: ",f1_score)

Accuracy:  0.696
Precision:  0.8533333333333334
Recall:  0.7032967032967034
F1 Score:  0.7710843373493976
